In [89]:
import os 

In [90]:
%pwd

'/'

In [92]:
os.chdir('/home/user/Desktop/data science/kidney-disease-classification-using-mlflow--End-to-End-Project')

In [93]:
%pwd

'/home/user/Desktop/data science/kidney-disease-classification-using-mlflow--End-to-End-Project'

In [94]:
from dataclasses import dataclass
from pathlib import Path



@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file : Path
    unzip_dir : Path

In [95]:
from kidneyDisease.constants import *
from kidneyDisease.utils.common import read_yaml,create_directories

In [96]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config 


In [97]:
import os 
import zipfile
import gdown
from kidneyDisease import logger
from kidneyDisease.utils.common import get_size


In [98]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [99]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-01 18:48:42,837: INFO: common: yaml file: config/config.yaml loaded successfully]


[2024-08-01 18:48:42,842: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-01 18:48:42,845: INFO: common: created directory at: artifacts]
[2024-08-01 18:48:42,847: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-01 18:48:42,848: INFO: 986331348: Downloading data from https://drive.google.com/file/d/1w0uM3pK-bXDkGEe1kRTuObX_ThC5ZciK/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1w0uM3pK-bXDkGEe1kRTuObX_ThC5ZciK
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1w0uM3pK-bXDkGEe1kRTuObX_ThC5ZciK&confirm=t&uuid=7c17c9d7-dc3e-4624-ac77-a6df65892924
To: /home/user/Desktop/data science/kidney-disease-classification-using-mlflow--End-to-End-Project/artifacts/data_ingestion/data.zip
100%|██████████| 57.7M/57.7M [01:11<00:00, 812kB/s] 

[2024-08-01 18:49:59,991: INFO: 986331348: Downloaded data from https://drive.google.com/file/d/1w0uM3pK-bXDkGEe1kRTuObX_ThC5ZciK/view?usp=sharing into file artifacts/data_ingestion/data.zip]
